In [1]:
! pip install langchain unstructured[all-docs] pydantic lxml openai chromadb tiktoken

  Using cached lxml-5.2.2-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---- --------------------------------- 122.9/981.5 kB 7.5 MB/s eta 0:00:01
     --------- ---------------------------- 235.5/981.5 kB 4.9 MB/s eta 0:00:01
     ----------- -------------------------- 307.2/981.5 kB 3.8 MB/s eta 0:00:01
     --------------- ---------------------- 389.1/981.5 kB 3.5 MB/s eta 0:00:01
     ----------------- -------------------- 460.8/981.5 kB 2.6 MB/s eta 0:00:01
     ----------------------- -------------- 614.4/981.5 kB 3.0 MB/s eta 0:00:01
     -------------------------- ----------- 686.1/981.5 kB 2.9 MB/s eta 0:00:01
     -------------------------------- ----- 829.4/981.5 kB 2.8 MB/s eta 0:00:01
     ---------------------------------- --- 890.9/981.5 kB 2.7 MB/s eta 0:00:01
     -------


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Data Loading

### Partition PDF tables, text, and images

* Use [Unstructured](https://unstructured-io.github.io/unstructured/) to partition elements

In [1]:
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

# Get elements
raw_pdf_elements = partition_pdf(
    filename= "quyche.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
)

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 19,
 "<class 'unstructured.documents.elements.Table'>": 3}

In [34]:
import base64

text_elements = []
table_elements = []
image_elements = []

# Function to encode images
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

for element in raw_pdf_elements:
    if 'CompositeElement' in str(type(element)):
        text_elements.append(element)
    elif 'Table' in str(type(element)):
        table_elements.append(element)

table_elements = [i.text for i in table_elements]
text_elements = [i.text for i in text_elements]

# Tables
print(len(table_elements))

# Text
print(len(text_elements))

3
19


## Multi-vector retriever

Use [multi-vector-retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector#summary).

Summaries are used to retrieve raw tables and / or raw chunks of text.

### Add to vectorstore

Use [Multi Vector Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector#summary) with summaries.

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os
api_key = os.environ["OPENAI_API_KEY"]

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage

chain_gpt_35 = ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=1024, api_key=api_key)
# chain_gpt_4_vision = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)

# Function for text summaries
def summarize_text(text_element):
    prompt = f"Tóm tắt đoạn văn sau:\n\n{text_element}\n\nTóm tắt:"
    response = chain_gpt_35.invoke([HumanMessage(content=prompt)])
    return response.content

# Function for table summaries
def summarize_table(table_element):
    prompt = f"Tóm tắt bảng sau:\n\n{table_element}\n\nTóm tắt:"
    response = chain_gpt_35.invoke([HumanMessage(content=prompt)])
    return response.content

# Function for image summaries
# def summarize_image(encoded_image):
#     prompt = [
#         AIMessage(content="You are a bot that is good at analyzing images."),
#         HumanMessage(content=[
#             {"type": "text", "text": "Describe the contents of this image."},
#             {
#                 "type": "image_url",
#                 "image_url": {
#                     "url": f"data:image/jpeg;base64,{encoded_image}"
#                 },
#             },
#         ])
#     ]
#     response = chain_gpt_4_vision.invoke(prompt)
#     return response.content

In [39]:
# Processing table elements with feedback and sleep
table_summaries = []
for i, te in enumerate(table_elements):
    summary = summarize_table(te)
    table_summaries.append(summary)
    print(summary)

Bảng này mô tả cách điểm số được chuyển đổi từ thang điểm 10 sang thang điểm 4. Điểm số từ 9,5 đến 10 sẽ được chuyển thành A+, từ 8,5 đến 9,4 chuyển thành A, từ 8,0 đến 8,4 chuyển thành B+ và tiếp tục như vậy. Điểm chữ tương ứng với điểm số được xác định dựa trên khoảng điểm số nằm trong. Không đạt sẽ được gán điểm số là 0.
Bảng trình độ học vị và số tín chỉ tích lũy của sinh viên trong các năm học. Sinh viên năm thứ nhất dưới 32 tín chỉ, sinh viên năm thứ hai từ 32 đến dưới 64 tín chỉ, sinh viên năm thứ ba từ 64 đến dưới 96 tín chỉ, sinh viên năm thứ tư từ 96 đến dưới 128 tín chỉ, và sinh viên năm thứ năm từ 128 tín chỉ trở lên.
Bảng trên phân loại học lực của sinh viên dựa trên điểm trung bình tích lũy. Sinh viên được xếp loại từ xuất sắc đến kém dưới 1,0 tùy theo khoảng điểm trung bình tích lũy của họ.


In [61]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.schema.document import Document
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma
import uuid

# Initialize the vector store and storage layer
vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings(api_key=api_key))
store = InMemoryStore()
id_key = "doc_id"
# Initialize the retriever
retriever = MultiVectorRetriever(vectorstore=vectorstore, docstore=store, id_key=id_key)

# Function to add documents to the retriever
def add_documents_to_retriever(summaries, original_contents):
    doc_ids = [str(uuid.uuid4()) for _ in summaries]
    summary_docs = [
        Document(page_content=s, metadata={id_key: doc_ids[i]})
        for i, s in enumerate(summaries)
    ]
    retriever.vectorstore.add_documents(summary_docs)
    retriever.docstore.mset(list(zip(doc_ids, original_contents)))

In [62]:
add_documents_to_retriever(table_summaries, table_elements)
vectorstore.persist()

# Table retrieval

The most complex table in the paper:

In [ ]:
persit_directory = "vectorstore"
vectorstore = Chroma(collection_name="summaries", 
                     embedding_function=OpenAIEmbeddings(api_key=api_key),
                     persist_directory=persit_directory)

retriever = MultiVectorRetriever(vectorstore=vectorstore, docstore=store, id_key=id_key)

retriever.get_relevant_documents(
    "tích lũy được 50 tín chỉ là sinh viên năm mấy?"
)

We can retrieve this image summary:

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

template = """Answer the question based only on the following context, which can include text, images and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke(
     "What do you see on the images in the database?"
)